Mainly serve as a template for future strategies. The template use simple moving average (SMA) of different timeframes to generate buy and sell signals. The higher timeframe SMA is used to generate the trend, and the lower timeframe SMA is used to generate the entry and exit points.

In [1]:
from typing_extensions import Dict
import optuna
import pandas as pd
import numpy as np
import talib as ta
# --- Our Imports ---
from AlgoTrade.Utils.DataManager import DataManager
from AlgoTrade.Config.BacktestConfig import BacktestConfig
from AlgoTrade.Backtester.BacktestRunner import BacktestRunner
from AlgoTrade.Factories.IndicatorFactory import IndicatorFactory
from AlgoTrade.Optimizer.StudyRunner import run_optimization

In [2]:
dm = DataManager(name="bitget")
ohlcv_data_s = dm.from_local("ADA/USDT:USDT", "15m", "2024-12-01")
ohlcv_data_l = dm.from_local("ADA/USDT:USDT", "1h", "2025-01-01")

Loading data from d:\ComputerScience\Trading\Quant2\data\bitget\ADA_USDT_USDT_15m.csv...
Load completed successfully. Data shape: (16960, 6)
Loading data from d:\ComputerScience\Trading\Quant2\data\bitget\ADA_USDT_USDT_1h.csv...
Load completed successfully. Data shape: (3972, 5)


d:\ComputerScience\Trading\Quant2\AlgoTrade\Utils\DataManager.py:289: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data.index = pd.to_datetime(data.index)


In [3]:
def pre_process_data(df1: pd.DataFrame, df2: pd.DataFrame):
    df1["datetime"] = df1.index
    df1.reset_index(drop=True, inplace=True)
    df2["datetime"] = df2.index
    df2.reset_index(drop=True, inplace=True)
    print("Data preprocessed.")
    return df1, df2

In [4]:
def merge_htf_into_ltf(df_ltf, df_htf, suffix):
    df_ltf = df_ltf.copy()
    df_htf = df_htf.copy()
    df_ltf["datetime"] = pd.to_datetime(df_ltf["datetime"])
    df_htf["datetime"] = pd.to_datetime(df_htf["datetime"])
    df_ltf.set_index("datetime", inplace=True)
    df_htf.set_index("datetime", inplace=True)
    df_htf = df_htf.shift(1)
    df_htf_resampled = df_htf.reindex(df_ltf.index, method="ffill")
    df_merged = df_ltf.join(df_htf_resampled, rsuffix=suffix)
    df_merged.dropna(inplace=True)
    return df_merged.reset_index()

In [5]:
df1, df2 = pre_process_data(ohlcv_data_s, ohlcv_data_l)
ltf_factory = IndicatorFactory(df1)
htf_factory = IndicatorFactory(df2)
df1 = ltf_factory.add_ema(20).add_ema(10).get_data()
df2 = htf_factory.add_ema(20).add_ema(10).get_data()

Data preprocessed.


In [ ]:
merge_df = merge_htf_into_ltf(df1, df2, suffix="_4h")


In [ ]:
def compute_signals(merged_df: pd.DataFrame):
    signal = np.where(
        (merged_df["trend"] == 1)
        & (merged_df["close"] > merged_df["MA"])
        & (merged_df["SD_ATR_Spread"] > 0),
        1,
        np.where(
            (merged_df["trend"] == -1)
            & (merged_df["close"] < merged_df["MA"])
            & (merged_df["SD_ATR_Spread"] > 0),
            -1,
            0,
        ),
    )
    merged_df["signal"] = pd.Series(signal, index=merged_df.index).shift(1).fillna(0)
    return merged_df

In [ ]:
solution = compute_signals(merge_df)

# Set datetime to index to use the backtest

In [ ]:
solution.set_index('datetime', inplace=True)

# Select Position Sizing Method

In [ ]:
config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=10,
    position_sizing_method="PercentBalance",
    percent_balance_pct=0.1,  # <-- UPDATED
    exit_on_signal_0=True,
    # General SL/TP can be used as a fallback
    stop_loss_pct=0.02,
    take_profit_pct=0.02,
)

In [ ]:
runner = BacktestRunner(config=config, data=solution)
analysis = runner.run()
analysis.print_metrics()

# Select All Position Sizing Method

In [ ]:
base_config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=10,
    
    exit_on_signal_0=True,
    # --- Parameters for all sizing methods ---
    # For PercentBalance
    percent_balance_pct=0.1,
    # For FixedAmount
    stop_loss_pct=0.02,
    take_profit_pct=0.02,
    # For AtrVolatility
    atr_volatility_risk_pct=0.02,
    atr_volatility_period=14,
    atr_volatility_multiplier=2.5,
    # For KellyCriterion
    kelly_criterion_lookback=50,
    kelly_criterion_fraction=0.5,
    # For AtrBands
    atr_bands_risk_pct=0.02,
    atr_bands_period=14,
    atr_bands_multiplier=2.0,
    atr_bands_risk_reward_ratio=1.5,
)

In [ ]:
from AlgoTrade.Backtester import ComparativeRunner as cr

In [ ]:
res = cr.run_comparative_analysis(base_config, solution)

In [ ]:
cr.print_comparison_report(res)

# Optimizer



In [ ]:
def generate_signals_for_trial(
    trial: optuna.trial.Trial, data_dict: Dict[str, pd.DataFrame]
) -> pd.DataFrame:
    """
    This function is passed to the optimizer. It defines the parameters to tune
    and returns a DataFrame with a 'signal' column.
    """
    # 1. Define the parameters you want to optimize for this strategy
    ltf_ma_period = trial.suggest_int("ltf_ma_period", 20, 100, step=5)
    htf_ma_period = trial.suggest_int("htf_ma_period", 5, 50, step=1)
    # You could also tune other things, e.g., a volatility filter threshold
    # min_spread = trial.suggest_float("min_spread", -0.0001, 0.0001)

    # 2. Get the data
    df1, df2 = pre_process_data(
        data_dict["ltf_data"].copy(), data_dict["htf_data"].copy()
    )

    # 3. Calculate indicators using the trial's parameters
    df1 = compute_indicators_df1(df1, period_df1=ltf_ma_period)
    df2 = compute_indicators_df2(df2, period_df2=htf_ma_period)

    # 4. Merge and compute final signal
    merged_df = merge_htf_into_ltf(df1, df2, suffix="_4h")
    solution_df = compute_signals(merged_df)

    # 5. Set index and return
    solution_df.set_index("datetime", inplace=True)
    return solution_df

In [ ]:
data_dictionary = {
    "ltf_data": ohlcv_data_s,
    "htf_data": ohlcv_data_l,
}

fixed_config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=10,
    position_sizing_method="AtrBands",
    atr_bands_risk_pct=0.01,
    atr_bands_period=14,
    atr_bands_multiplier=2.0,
    atr_bands_risk_reward_ratio=1.5,
    exit_on_signal_0=False,
    allow_reverse_trade=False,
)

In [ ]:
print("\n--- Running Strategy Optimization ---")
run_optimization(
    data_dict=data_dictionary,
    config=fixed_config,
    strategy_function=generate_signals_for_trial,
    n_trials=10,
    metric_to_optimize="sharpe_ratio"
)
